In [1]:
import pandas as pd
import numpy as np

In [2]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder

In [3]:
df = pd.read_csv('Datasets/covid_toy.csv')
df.head()

,age,gender,fever,cough,city,has_covid
0,60,Male,103.0,Mild,Kolkata,No
1,27,Male,100.0,Mild,Delhi,Yes
2,42,Male,101.0,Mild,Delhi,No
3,31,Female,98.0,Mild,Kolkata,No
4,65,Female,101.0,Mild,Mumbai,No


In [4]:
df['cough'].value_counts()

Mild      62
Strong    38
Name: cough, dtype: int64

In [5]:
df['city'].value_counts()

Kolkata      32
Bangalore    30
Delhi        22
Mumbai       16
Name: city, dtype: int64

In [6]:
df.isna().sum()

age           0
gender        0
fever        10
cough         0
city          0
has_covid     0
dtype: int64

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test,y_trian,y_test = train_test_split(df.drop(columns=['has_covid'],axis=1),df['has_covid'],test_size=0.2,random_state=0,shuffle=True)

In [8]:
X_train

,age,gender,fever,cough,city
43,22,Female,99.0,Mild,Bangalore
62,56,Female,104.0,Strong,Bangalore
3,31,Female,98.0,Mild,Kolkata
71,75,Female,104.0,Strong,Delhi
45,72,Male,99.0,Mild,Bangalore
...,...,...,...,...,...
96,51,Female,101.0,Strong,Kolkata
67,65,Male,99.0,Mild,Bangalore
64,42,Male,104.0,Mild,Mumbai
47,18,Female,104.0,Mild,Bangalore


## Manually Handling the Features

In [ ]:
si = SimpleImputer()
X_train_fever = si.fit_transform(X_train[['fever']])
X_test_fever = si.transform(X_test[['fever']])
X_train_fever.shape

## Handling Nominal columns

In [ ]:
df.head()

In [ ]:
#gender, city 
ohe = OneHotEncoder(sparse_output=False,drop='first',dtype=np.int32)
ohe.fit(X_train[['gender']])
X_train_gender = ohe.transform(X_train[['gender']])
X_test_gender = ohe.transform(X_test[['gender']])
X_train_gender

In [ ]:
ohe.categories_

In [ ]:
ohe.fit(X_train[['city']])
X_train_city = ohe.transform(X_train[['city']])
X_test_city = ohe.transform(X_test[['city']])
X_train_city

In [ ]:
ohe.categories_

# Ordinal Encoding 

In [ ]:
df['cough'].value_counts()


In [ ]:
ode = OrdinalEncoder(categories=[['Mild','Strong']],dtype=np.int32)

In [ ]:
ode.fit(X_train[['cough']])
X_train_cough = ode.transform(X_train[['cough']])
X_test_cough = ode.transform(X_test[['cough']])
X_test_cough

In [ ]:
ode.categories_

In [ ]:
X_train['gender'] = X_train_gender

In [ ]:
#X_train['city'] = X_train_city
#X_test['city'] = X_test_city
X_test['gender'] = X_test_gender
X_train['cough'] = X_train_cough
X_test['cough'] = X_test_cough

In [ ]:
X_train

In [ ]:
new_dframe = pd.DataFrame(np.hstack((X_train.drop(['city'],axis=1).values,X_train_city)),columns=['age','gender','fever','cough','First_city','second_city','Third_city'])
new_dframe

## All stuff using Column Tranformer 

In [16]:
from sklearn.compose import ColumnTransformer

In [17]:
transformer = ColumnTransformer(transformers=[
    ('tnf1',SimpleImputer(),['fever']),
    ('tnf2',OrdinalEncoder(categories=[['Mild','Strong']]),['cough']),
    ('tnf3',OneHotEncoder(sparse_output=False,drop='first'),['gender','city'])
],remainder='passthrough')

In [20]:
new_df = pd.DataFrame(transformer.fit_transform(X_train))

In [21]:
new_df

,0,1,2,3,4,5,6
0,99.0,0.0,0.0,0.0,0.0,0.0,22.0
1,104.0,1.0,0.0,0.0,0.0,0.0,56.0
2,98.0,0.0,0.0,0.0,1.0,0.0,31.0
3,104.0,1.0,0.0,1.0,0.0,0.0,75.0
4,99.0,0.0,1.0,0.0,0.0,0.0,72.0
...,...,...,...,...,...,...,...
75,101.0,1.0,0.0,0.0,1.0,0.0,51.0
76,99.0,0.0,1.0,0.0,0.0,0.0,65.0
77,104.0,0.0,1.0,0.0,0.0,1.0,42.0
78,104.0,0.0,0.0,0.0,0.0,0.0,18.0
